In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig
from tqdm import tqdm

import sys
sys.path.append("../src")

from data_prep import *
from make_cuts import *
from fitting_functions import *
from pulse_profile_generator import *
from plot_maker import *
from pulse_plot_maker import *

In [2]:
pd.set_option('display.max_columns', None)

In [10]:
ci_folder="../../data/calibration/charge_injections/"
c_folder="../../data/calibration/"
Path("../../plots").mkdir(parents=True, exist_ok=True)
Path("../../plots/charge_injections").mkdir(parents=True, exist_ok=True)
p_folder="../../plots/charge_injections/"

In [6]:
charge_injection_runs=["scan_DPM0_CALIBRUN_coff14_20220419_073825.csv",
                       "scan_DPM0_CALIBRUN_coff14_20220424_071302.csv",
                       "scan_DPM0_CALIBRUN_coff14_20220424_220633.csv",
                       "scan_DPM1_CALIBRUN_coff14_20220419_081316.csv",
                       "scan_DPM1_CALIBRUN_coff14_20220424_071550.csv",
                       "scan_DPM1_CALIBRUN_coff14_20220424_220543.csv"]

In [36]:
c1=pd.read_csv(ci_folder+charge_injection_runs[0])
c2=pd.read_csv(ci_folder+charge_injection_runs[1])
c3=pd.read_csv(ci_folder+charge_injection_runs[3])

In [11]:
connections=pd.read_csv(c_folder+"testbeam_connections.csv")

In [25]:
connections

,Polarfire,HGCROC,Channel,CMB,Quadbar,Bar,Plane,Cable
0,0,0,0,1,1,1,1,11
1,0,0,1,1,1,2,1,11
2,0,0,2,1,1,3,1,11
3,0,0,3,1,1,4,1,11
4,0,0,4,2,1,1,1,12
...,...,...,...,...,...,...,...,...
379,1,2,66,15,3,4,19,195
380,1,2,67,16,3,1,19,196
381,1,2,68,16,3,2,19,196
382,1,2,69,16,3,3,19,196


In [35]:
# c1.merge(connections, how="inner", left_on=["CHAN", "DPM"], right_on=["Channel", "Polarfire"])
c1

,CALIB_DAC,DPM,ILINK,CHAN,EVENT,ADC0,ADC1,ADC2,ADC3,ADC4,ADC5,ADC6,ADC7,TOT0,TOT1,TOT2,TOT3,TOT4,TOT5,TOT6,TOT7,TOA0,TOA1,TOA2,TOA3,TOA4,TOA5,TOA6,TOA7,CAPACITOR_TYPE,SIPM_BIAS
0,0,0,0,0,0,93,94,101,101,99,99,95,93,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3784
1,0,0,1,0,0,103,103,107,114,113,110,106,105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3784
2,0,0,2,0,0,97,101,106,116,118,111,106,100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3784
3,0,0,3,0,0,104,105,111,115,116,111,107,106,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3784
4,0,0,4,0,0,95,95,98,111,115,119,115,111,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3784
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64795,2047,0,1,35,2,106,105,1023,1023,1023,1023,1023,1023,0,0,2410,0,0,0,0,0,0,0,809,0,0,0,0,0,1,3784
64796,2047,0,2,35,2,94,94,1023,1023,1023,1023,1023,1023,0,0,2383,0,0,0,0,0,0,0,319,0,0,0,0,0,1,3784
64797,2047,0,3,35,2,99,101,1023,1023,1023,1023,1023,1023,0,0,2326,0,0,0,0,0,0,0,342,0,0,0,0,0,1,3784
64798,2047,0,4,35,2,103,103,1023,1023,1023,1023,1023,1023,0,0,2363,0,0,0,0,0,0,0,92,0,0,0,0,0,1,3784


In [27]:
c3

,CALIB_DAC,DPM,ILINK,CHAN,EVENT,ADC0,ADC1,ADC2,ADC3,ADC4,ADC5,ADC6,ADC7,TOT0,TOT1,TOT2,TOT3,TOT4,TOT5,TOT6,TOT7,TOA0,TOA1,TOA2,TOA3,TOA4,TOA5,TOA6,TOA7,CAPACITOR_TYPE,SIPM_BIAS
0,0,1,0,0,0,99,95,118,126,107,97,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3784
1,0,1,1,0,0,111,107,127,131,115,106,105,107,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3784
2,0,1,2,0,0,103,102,109,115,115,112,109,105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3784
3,0,1,3,0,0,95,94,100,105,105,103,97,49,0,0,0,0,0,0,0,2048,0,0,0,0,0,0,0,0,0,3784
4,0,1,4,0,0,94,94,98,104,104,102,94,94,0,0,0,0,0,16,16,16,64,64,64,64,64,64,64,64,0,3784
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64795,2047,1,1,35,2,108,109,1023,1023,1023,1023,1023,1023,0,0,2346,0,0,0,0,0,0,0,810,0,0,0,0,0,1,3784
64796,2047,1,2,35,2,101,103,1023,1023,1023,1023,1023,1023,0,0,2366,0,0,0,0,0,0,0,325,0,0,0,0,0,1,3784
64797,2047,1,3,35,2,98,99,1023,511,511,511,511,511,0,0,2234,2048,2048,2048,0,0,0,0,178,0,0,0,0,0,1,3784
64798,2047,1,4,35,2,96,94,766,766,766,766,766,766,0,0,2216,0,0,0,0,0,64,64,155,64,0,64,0,64,1,3784


In [37]:
c2

,CALIB_DAC,DPM,ILINK,CHAN,EVENT,ADC0,ADC1,ADC2,ADC3,ADC4,ADC5,ADC6,ADC7,TOT0,TOT1,TOT2,TOT3,TOT4,TOT5,TOT6,TOT7,TOA0,TOA1,TOA2,TOA3,TOA4,TOA5,TOA6,TOA7,CAPACITOR_TYPE,SIPM_BIAS
0,0,0,0,0,0,93,95,99,102,102,99,95,95,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3784
1,0,0,1,0,0,105,103,108,113,111,111,110,107,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3784
2,0,0,2,0,0,97,95,102,111,113,111,104,95,0,0,0,0,0,0,0,0,0,0,0,0,225,0,0,0,0,3784
3,0,0,3,0,0,103,103,109,111,110,109,109,105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3784
4,0,0,4,0,0,100,98,103,114,117,116,111,109,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3784
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64795,2047,0,1,35,2,106,105,1023,1023,1023,1023,1023,1023,0,0,2412,0,0,0,0,0,0,0,810,0,0,0,0,0,1,3784
64796,2047,0,2,35,2,93,93,1023,1023,1023,1023,1023,1023,0,0,2373,0,0,0,0,0,0,0,334,0,0,0,0,0,1,3784
64797,2047,0,3,35,2,99,101,1023,1023,1023,1023,1023,1023,0,0,2327,0,0,0,0,0,0,0,341,0,0,0,0,0,1,3784
64798,2047,0,4,35,2,103,103,1023,1023,1023,1023,1023,1023,0,0,2378,0,0,0,0,0,0,0,68,0,0,0,0,0,1,3784
